In [2]:
%run ../../utils/commonImports.py
%run ../../utils/tradingImports.py
%matplotlib inline

# Load Data

In [3]:
file = 'D:\\Dropbox\\My work\\krypl-project\\dataSplit\\01-priceMovement\\strategy_test.tsv'
data = read_tsv(file)
features = ['return', 'return_1', 'return_2', 'zscore_36_return',
       'zscore_288_return', 'zscore_36_volume', 'zscore_288_volume',
       'zscore_2016_volume', 'pma36', 'pma288', 'pma2016', 'vma36', 'vma288',
       'vma2016', 'ma_12_108', 'ma_36_288', 'ac36', 'ac288', 'vol36', 'vol288',
       'vol2016', 'dv_36_288', 'dv_288_2016']
data_manager = CurrencyDataManager(data['close'], data[features])
wallet = {'usdt': 1000}
contract_pair = ContractPair.new('usdt', 'btc')

# Load Model

In [4]:
data['date'].min()

'2017-10-05 00:00:00'

In [5]:
data['date'].max()

'2018-10-05 13:15:00'

In [6]:
from sklearn.externals import joblib
from modeling.gridsearch import GridSearch

path = "D:\\Dropbox\\My work\\krypl-project\\models\\01-priceMovement"
clf = load_model(f'{path}\\tree_selected_01.pkl')

# Implementation

In [7]:
from modeling.strategy import Strategy

class HoldStrategy(Strategy):
    def __init__(self, exchange, data_manager, contract_pair, trade_size):
        super().__init__(exchange, data_manager, contract_pair, trade_size, 0., 0.)    
    
    def trade(self):
        while self.data_manager.has_tick():            
            history, price = self.data_manager.tick(1)
            
            if history.shape[0] == 0:
                self.buy(price)
            else:
                continue
        
        if self.opened:
            self.sell_all(price)

In [20]:
from modeling.strategy import Strategy


class MachineLearningStrategy(Strategy):
    def __init__(self, exchange, data_manager, contract_pair, trade_size, willing_loss, target_profit, clf, max_hold_time):
        super().__init__(exchange, data_manager, contract_pair, trade_size, willing_loss, target_profit)
        self.history_len = 1
        self.max_hold_time = max_hold_time
        self.clf = clf
        self.consecutive = 0
        self.last_prediction = 0
    
    def should_buy(self, history):
        if self.trade_size >= self.exchange.wallet[contract_pair['priceContract']] * 1.1:
            return False
        
        if bool(self.last_prediction):
            self.consecutive += 1
        else:
            self.consecutive = 0
        
        self.last_prediction = self.clf.predict(history)[0]
        return bool(self.last_prediction) and self.consecutive > 10
    
    def should_sell(self, hold):
        return hold >= self.max_hold_time
    
    
    def trade(self):
        after_sell = False
        hold = 0
        while self.data_manager.has_tick():            
            history, price = self.data_manager.tick(self.history_len)
            
            if history.shape[0] == 0:
                continue
            
            if not self.opened and self.should_buy(history):
                self.buy(price)
                price_bought = price
                hold = 0
            elif self.opened and self.is_risky(price_bought, price):
                self.sell_all(price)
                after_sell = True
                hold = 0
            elif self.opened and self.is_target_satisfied(price_bought, price):
                self.sell_all(price)
                after_sell = True
                hold = 0
            elif self.opened and self.should_sell(hold):
                self.sell_all(price)
                after_sell = True
                hold = 0
            
            if self.opened:
                hold += 1
        
        if self.opened:
            self.sell_all(price)

In [18]:
def hold_stats(data_manager_p):
    data_manager = deepcopy(data_manager_p)
    exchange = BackTestExchange(data_manager, deepcopy(wallet), 0.0025)
    strategy = HoldStrategy(exchange, data_manager, contract_pair, 100)
    strategy.trade()
    return strategy.stats('usdt').report()

# Trading

In [14]:
hold_stats_train = hold_stats(data_manager)
hold_stats_train

,usdt
startAmount,"1,000.0000"
numberOfTrades,1.0000
totalProfit,55.8379
avgProfit,55.8379
winPercentage,100.0000
avgWinTrade,55.8379
avgLossTrade,0.0000
profitFactor,inf
maxDrawdown,0.0000


In [21]:
DAY = (60 / 5) * 24
dm = deepcopy(data_manager)
exchange = BackTestExchange(dm, deepcopy(wallet), 0.0025)
strategy = MachineLearningStrategy(exchange, dm, contract_pair, 100, willing_loss=0.05, target_profit=0.01, clf=clf, max_hold_time=DAY)
strategy.trade()
r = strategy.stats('usdt').report()
r

,usdt
startAmount,"1,000.0000"
numberOfTrades,373.0000
totalProfit,-160.2617
avgProfit,-0.4297
winPercentage,65.6836
avgWinTrade,1.3147
avgLossTrade,-3.7684
profitFactor,0.6678
maxDrawdown,19.0398


# Look on transactions

In [11]:
len(strategy.exchange.transactions)

1468

In [ ]:
%matplotlib notebook
plot_transactions(data.copy(), strategy.exchange.transactions[1100:1110])

# Look on model predictions

In [ ]:
look_data = data.copy()
look_data['prediction'] = clf.predict(data[features])

restricted = look_data.iloc[:, :].reset_index()
plot_candles(restricted)

trues = restricted.query('prediction == 1')
plt.scatter(trues.index, trues['prediction'] * trues['close']);